In [ ]:
# 在Jupyter中内联显示matplotlib图表
%matplotlib inline
import torch  # PyTorch深度学习框架
import torchvision  # PyTorch的计算机视觉库，提供图像处理功能
from torch import nn  # 神经网络模块
from d2l import torch as d2l  # 《动手学深度学习》工具库
from torch.nn import functional as F  # 神经网络函数库，包含各种激活函数和损失函数

In [ ]:
# 类别预测器：预测每个锚框包含各个类别的概率
def cls_predictor(num_inputs, num_anchors, num_classes):
    """
    参数:
        num_inputs: 输入特征图的通道数
        num_anchors: 每个像素生成的锚框数量
        num_classes: 目标类别数（不包括背景）
    返回:
        卷积层，输出通道数 = 锚框数 × (类别数 + 1)
        +1 是因为还要预测背景类
    """
    return nn.Conv2d(
        num_inputs,  # 输入通道数
        num_anchors * (num_classes + 1),  # 输出通道数：每个锚框预测所有类别+背景的概率
        kernel_size=3,  # 3x3卷积核
        padding=1  # padding=1保持特征图尺寸不变
    )

In [ ]:
# 边界框预测器：预测每个锚框的偏移量
def bbox_predictor(num_inputs, num_anchors):
    """
    参数:
        num_inputs: 输入特征图的通道数
        num_anchors: 每个像素生成的锚框数量
    返回:
        卷积层，输出通道数 = 锚框数 × 4
        4 表示边界框的4个坐标值 (x, y, w, h)
    """
    return nn.Conv2d(
        num_inputs,  # 输入通道数
        num_anchors * 4,  # 输出通道数：每个锚框预测4个偏移量
        kernel_size=3,  # 3x3卷积核
        padding=1  # padding=1保持特征图尺寸不变
    )

In [ ]:
# 前向传播辅助函数：将输入通过网络块
def forward(x, block):
    """
    参数:
        x: 输入张量
        block: 网络模块
    返回:
        经过block处理后的输出
    """
    return block(x)

In [ ]:
# 测试类别预测器在不同特征图尺寸下的输出
# Y1: 模拟第一个特征层的输出
# 输入: (2, 8, 20, 20) - 批量2，8通道，20x20特征图
# 每个位置5个锚框，10个类别+1个背景 = 11
# 输出形状: (2, 5*11, 20, 20) = (2, 55, 20, 20)
Y1 = forward(torch.zeros((2, 8, 20, 20)), cls_predictor(8, 5, 10))

# Y2: 模拟第二个特征层的输出
# 输入: (2, 16, 10, 10) - 批量2，16通道，10x10特征图
# 每个位置3个锚框，10个类别+1个背景 = 11
# 输出形状: (2, 3*11, 10, 10) = (2, 33, 10, 10)
Y2 = forward(torch.zeros((2, 16, 10, 10)), cls_predictor(16, 3, 10))

print(Y1.shape)  # 打印第一个特征层的输出形状
print(Y2.shape)  # 打印第二个特征层的输出形状

torch.Size([2, 55, 20, 20])
torch.Size([2, 33, 10, 10])


In [ ]:
# 展平预测结果：将预测张量重组为二维
def flatten_pred(pred):
    """
    将预测张量从 (batch, channels, height, width) 
    转换为 (batch, height*width*channels_per_anchor)
    """
    # permute(0,2,3,1): 将通道维度移到最后 (batch, height, width, channels)
    # flatten(start_dim=1): 从第1维开始展平，保留batch维度
    return torch.flatten(pred.permute(0, 2, 3, 1), start_dim=1)

# 拼接多个特征层的预测结果
def concat_preds(preds):
    """
    将不同特征层的预测结果在锚框维度上拼接
    参数:
        preds: 预测列表，每个元素是一个特征层的预测
    返回:
        拼接后的张量，形状 (batch, total_anchors*predictions_per_anchor)
    """
    return torch.cat([flatten_pred(p) for p in preds], dim=1)

In [ ]:
# 测试拼接功能
# Y1形状: (2, 55, 20, 20) -> 展平后 (2, 20*20*55) = (2, 22000)
# Y2形状: (2, 33, 10, 10) -> 展平后 (2, 10*10*33) = (2, 3300)
# 拼接后: (2, 22000+3300) = (2, 25300)
print(concat_preds([Y1, Y2]).shape)

torch.Size([2, 25300])


In [ ]:
# 下采样块：用于减小特征图尺寸并增加感受野
def down_sample_blk(in_channels, out_channels):
    """
    参数:
        in_channels: 输入通道数
        out_channels: 输出通道数
    返回:
        包含2个卷积层、批量归一化、ReLU激活和最大池化的序列模块
    """
    blk = []  # 创建空列表存储层
    for _ in range(2):  # 重复2次：添加2个卷积块
        # 3x3卷积，padding=1保持特征图尺寸
        blk.append(nn.Conv2d(in_channels, out_channels,
                             kernel_size=3, padding=1))
        blk.append(nn.BatchNorm2d(out_channels))  # 批量归一化
        blk.append(nn.ReLU())  # ReLU激活函数
        in_channels = out_channels  # 更新输入通道数（第二个卷积的输入）
    # 2x2最大池化，步幅2，将特征图尺寸减半
    blk.append(nn.MaxPool2d(2))
    return nn.Sequential(*blk)  # 组合成序列模块

In [ ]:
# 测试下采样块
# 输入: (2, 3, 20, 20) - 批量2，3通道，20x20特征图
# 经过下采样块: 通道3->10，尺寸20x20->10x10（池化减半）
# 输出形状: (2, 10, 10, 10)
print(forward(torch.zeros((2, 3, 20, 20)), down_sample_blk(3, 10)).shape)

torch.Size([2, 10, 10, 10])


In [ ]:
# 基础网络：提取图像特征的主干网络
def base_net():
    """
    构建基础网络，包含3个下采样块
    通道变化: 3 -> 16 -> 32 -> 64
    尺寸变化: 每个下采样块将尺寸减半（共缩小8倍）
    """
    blk = []  # 创建空列表
    num_filters = [3, 16, 32, 64]  # 定义各层的通道数
    for i in range(len(num_filters) - 1):  # 遍历创建3个下采样块
        blk.append(
            down_sample_blk(num_filters[i], num_filters[i+1])  # 逐层增加通道数
        )
    return nn.Sequential(*blk)  # 组合成序列模块

In [ ]:
# 测试基础网络
# 输入: (2, 3, 256, 256) - 批量2，3通道（RGB），256x256图像
# 经过3个下采样块，每个将尺寸减半: 256 -> 128 -> 64 -> 32
# 通道数变化: 3 -> 16 -> 32 -> 64
# 输出形状: (2, 64, 32, 32)
print(forward(torch.zeros((2, 3, 256, 256)), base_net()).shape)

torch.Size([2, 64, 32, 32])


In [ ]:
# 获取第i个特征提取块：构建多尺度特征图
def get_blk(i):
    """
    根据索引返回不同的网络块，用于生成不同尺度的特征图
    参数:
        i: 块的索引 (0-4)
    返回:
        对应的网络块
    """
    if i == 0:
        return base_net()  # 第0块：基础网络，输出64通道
    elif i == 1:
        return down_sample_blk(64, 128)  # 第1块：64->128通道，尺寸减半
    elif i == 4:
        return nn.AdaptiveAvgPool2d((1, 1))  # 第4块：全局平均池化到1x1
    else:
        return down_sample_blk(128, 128)  # 第2,3块：保持128通道，尺寸减半
    return blk

In [ ]:
# 块的前向传播：在特征图上生成锚框并进行预测
def blk_forward(X, blk, size, ratio, cls_predictors, bbox_predictors):
    """
    参数:
        X: 输入特征图
        blk: 特征提取块
        size: 锚框尺寸列表
        ratio: 锚框宽高比列表
        cls_predictors: 类别预测器
        bbox_predictors: 边界框预测器
    返回:
        Y: 输出特征图
        anchor: 生成的锚框
        cls_preds: 类别预测
        bbox_preds: 边界框预测
    """
    Y = blk(X)  # 通过特征提取块得到新的特征图
    # 在特征图Y上生成锚框，使用指定的尺寸和宽高比
    anchor = d2l.multibox_prior(
        Y, sizes=size, ratios=ratio
    )
    cls_preds = cls_predictors(Y)  # 预测每个锚框的类别
    bbox_preds = bbox_predictors(Y)  # 预测每个锚框的偏移量
    return (Y, anchor, cls_preds, bbox_preds)

In [ ]:
# 定义5个特征层的锚框参数
# 锚框尺寸：从小到大，用于检测不同大小的目标
size = [
    [0.2, 0.272],   # 第1层：检测小目标，尺寸为图像的20%-27.2%
    [0.37, 0.447],  # 第2层：检测中小目标，尺寸为图像的37%-44.7%
    [0.54, 0.619],  # 第3层：检测中等目标，尺寸为图像的54%-61.9%
    [0.71, 0.79],   # 第4层：检测中大目标，尺寸为图像的71%-79%
    [0.88, 0.961]   # 第5层：检测大目标，尺寸为图像的88%-96.1%
]

# 锚框宽高比：每层使用相同的宽高比
ratio = [
    [1, 2, 0.5]  # 宽高比1:1（正方形）、2:1（宽）、0.5:1（高）
] * 5  # 所有5层使用相同的宽高比设置

# 计算每个像素位置的锚框数量
# 每个位置：len(size[0])个尺寸 + len(ratio[0])个宽高比 - 1（避免重复）
# = 2 + 3 - 1 = 4个锚框
num_anchors = len(size[0]) + len(ratio[0]) - 1

In [ ]:
# TinySSD网络：简化版的SSD目标检测网络
class TinySSD(nn.Module):
    def __init__(self, num_classes, **kwargs):
        """
        初始化TinySSD网络
        参数:
            num_classes: 目标类别数（不包括背景）
        """
        super(TinySSD, self).__init__(**kwargs)  # 调用父类初始化
        self.num_classes = num_classes  # 保存类别数
        idx_to_in_channels = [64, 128, 128, 128, 128]  # 5个特征层的输入通道数
        
        # 动态创建5个特征提取块和对应的预测器
        for i in range(5):
            # 使用setattr动态设置属性
            # 创建第i个特征提取块，命名为 'blk_0', 'blk_1', ..., 'blk_4'
            setattr(
                self,
                f'blk_{i}',
                get_blk(i)
            )
            # 创建第i个类别预测器，命名为 'cls_0', 'cls_1', ..., 'cls_4'
            setattr(
                self,
                f'cls_{i}',
                cls_predictor(
                    idx_to_in_channels[i],  # 该层的输入通道数
                    num_anchors,  # 每个位置的锚框数量
                    num_classes  # 目标类别数
                )
            )
            # 创建第i个边界框预测器，命名为 'bbox_0', 'bbox_1', ..., 'bbox_4'
            setattr(
                self,
                f'bbox_{i}',
                bbox_predictor(
                    idx_to_in_channels[i],  # 该层的输入通道数
                    num_anchors  # 每个位置的锚框数量
                )
            )

    def forward(self, X):
        """
        前向传播
        参数:
            X: 输入图像张量
        返回:
            anchors: 所有锚框
            cls_preds: 所有类别预测
            bbox_preds: 所有边界框预测
        """
        # 初始化3个列表，每个包含5个None（对应5个特征层）
        anchors, cls_preds, bbox_preds = [None] * 5, [None] * 5, [None] * 5
        
        # 依次通过5个特征层
        for i in range(5):
            # 使用getattr动态获取属性
            # 获取当前特征图、锚框、类别预测和边界框预测
            X, anchors[i], cls_preds[i], bbox_preds[i] = blk_forward(
                X,  # 输入特征图
                getattr(self, f'blk_{i}'),  # 第i个特征提取块
                size[i],  # 第i层的锚框尺寸
                ratio[i],  # 第i层的锚框宽高比
                getattr(self, f'cls_{i}'),  # 第i个类别预测器
                getattr(self, f'bbox_{i}')  # 第i个边界框预测器
            )
        
        # 在锚框维度（dim=1）上拼接所有特征层的锚框
        anchors = torch.cat(anchors, dim=1)
        
        # 拼接所有特征层的类别预测
        cls_preds = concat_preds(cls_preds)
        # 重塑类别预测的形状：(batch, total_anchors, num_classes+1)
        cls_preds = cls_preds.reshape(
            cls_preds.shape[0], -1, self.num_classes + 1
        )
        
        # 拼接所有特征层的边界框预测
        bbox_preds = concat_preds(bbox_preds)
        
        # 返回锚框、类别预测和边界框预测
        return (anchors,
                cls_preds,
                bbox_preds
        )

In [ ]:
# 测试TinySSD网络的输出形状
net = TinySSD(num_classes=1)  # 创建网络，只有1个目标类别（香蕉）
X = torch.zeros((32, 3, 256, 256))  # 模拟输入：32张256x256的RGB图像
anchors, cls_preds, bbox_preds = net(X)  # 前向传播

# 输出各个预测的形状
print('output anchors shape:', anchors.shape)  # 锚框形状
print('output class preds shape:', cls_preds.shape)  # 类别预测形状
print('output bbox preds shape:', bbox_preds.shape)  # 边界框预测形状

output anchors shape: torch.Size([1, 5444, 4])
output class preds shape: torch.Size([32, 5444, 2])
output bbox preds shape: torch.Size([32, 21776])


In [ ]:
# 加载香蕉检测数据集
batch_size = 32  # 设置批量大小为32
# load_data_bananas: 加载香蕉数据集，返回训练和测试迭代器
# 只使用训练数据，忽略测试数据（用_表示）
train_iter, _ = d2l.load_data_bananas(batch_size)

read 1000 training examples
read 100 validation examples


In [ ]:
# 设置设备和优化器
device, net = d2l.try_gpu(), TinySSD(num_classes=1)  # 尝试使用GPU，创建网络
# SGD优化器：随机梯度下降
# lr=0.2: 学习率
# weight_decay=5e-4: 权重衰减（L2正则化），防止过拟合
trainer = torch.optim.SGD(net.parameters(), lr=0.2, weight_decay=5e-4)

In [ ]:
# 定义损失函数
cls_loss = nn.CrossEntropyLoss(reduction='none')  # 分类损失：交叉熵，不自动求平均
bbox_loss = nn.L1Loss(reduction='none')  # 边界框损失：L1损失（绝对值误差），不自动求平均

# 计算总损失：分类损失 + 边界框损失
def calc_loss(cls_preds, cls_labels, bbox_preds, bbox_labels, bbox_masks):
    """
    参数:
        cls_preds: 类别预测，形状 (batch, num_anchors, num_classes+1)
        cls_labels: 类别标签，形状 (batch, num_anchors)
        bbox_preds: 边界框预测，形状 (batch, num_anchors*4)
        bbox_labels: 边界框标签，形状 (batch, num_anchors*4)
        bbox_masks: 边界框掩码，形状 (batch, num_anchors*4)，标记哪些锚框需要计算边界框损失
    返回:
        总损失 = 分类损失 + 边界框损失
    """
    batch_size, num_classes = cls_preds.shape[0], cls_preds.shape[2]
    
    # 计算分类损失
    # reshape(-1, num_classes): 将所有锚框展平成二维 (batch*num_anchors, num_classes+1)
    # cls_labels.reshape(-1): 展平标签成一维 (batch*num_anchors)
    # 计算后reshape回 (batch, num_anchors)，然后对每个样本求平均
    cls = cls_loss(cls_preds.reshape(-1, num_classes),
                   cls_labels.reshape(-1)).reshape(batch_size, -1).mean(dim=1)
    
    # 计算边界框损失
    # bbox_preds * bbox_masks: 只计算正样本（包含目标的锚框）的损失
    # bbox_labels * bbox_masks: 对应的真实边界框
    # mean(dim=1): 对每个样本的所有坐标求平均
    bbox = bbox_loss(bbox_preds * bbox_masks,
                     bbox_labels * bbox_masks).mean(dim=1)
    
    return cls + bbox  # 返回总损失

# 分类准确率评估
def cls_eval(cls_preds, cls_labels):
    """
    计算分类准确的锚框数量
    """
    # argmax(dim=-1): 取最后一维（类别维度）的最大值索引，得到预测类别
    # 与真实标签比较，统计相等的数量
    return float((cls_preds.argmax(dim=-1).type(
        cls_labels.dtype) == cls_labels).sum())

# 边界框预测误差评估
def bbox_eval(bbox_preds, bbox_labels, bbox_masks):
    """
    计算边界框预测的绝对误差总和
    """
    # 计算预测和真实边界框的绝对误差，只考虑正样本（通过bbox_masks）
    return float((torch.abs((bbox_labels - bbox_preds) * bbox_masks)).sum())

In [ ]:
# 训练循环
num_epochs, timer = 20, d2l.Timer()  # 训练20个epoch，创建计时器
# 创建动画绘图器，用于可视化训练过程
animator = d2l.Animator(xlabel='epoch', xlim=[1, num_epochs],
                        legend=['class error', 'bbox mae'])
net = net.to(device)  # 将网络移到GPU/CPU

for epoch in range(num_epochs):  # 遍历每个epoch
    # 创建累加器，跟踪4个指标：
    # [0]分类正确数, [1]总锚框数, [2]边界框误差和, [3]边界框总数
    metric = d2l.Accumulator(4)
    net.train()  # 设置为训练模式
    
    for features, target in train_iter:  # 遍历每个批量
        timer.start()  # 开始计时
        trainer.zero_grad()  # 清空梯度
        X, Y = features.to(device), target.to(device)  # 将数据移到设备
        
        # 生成多尺度的锚框，为每个锚框预测类别和偏移量
        anchors, cls_preds, bbox_preds = net(X)
        
        # 为每个锚框标注类别和偏移量
        # multibox_target: 将真实目标分配给锚框，生成训练标签
        bbox_labels, bbox_masks, cls_labels = d2l.multibox_target(anchors, Y)
        
        # 根据类别和偏移量的预测和标注值计算损失函数
        l = calc_loss(cls_preds, cls_labels, bbox_preds, bbox_labels,
                      bbox_masks)
        
        l.mean().backward()  # 反向传播计算梯度
        trainer.step()  # 更新参数
        
        # 累加评估指标
        # cls_eval: 分类正确的锚框数
        # cls_labels.numel(): 总锚框数
        # bbox_eval: 边界框误差和
        # bbox_labels.numel(): 边界框坐标总数
        metric.add(cls_eval(cls_preds, cls_labels), cls_labels.numel(),
                   bbox_eval(bbox_preds, bbox_labels, bbox_masks),
                   bbox_labels.numel())
    
    # 计算分类错误率和边界框平均绝对误差
    cls_err, bbox_mae = 1 - metric[0] / metric[1], metric[2] / metric[3]
    animator.add(epoch + 1, (cls_err, bbox_mae))  # 添加到动画

# 打印最终结果
print(f'class err {cls_err:.2e}, bbox mae {bbox_mae:.2e}')
print(f'{len(train_iter.dataset) / timer.stop():.1f} examples/sec on '
      f'{str(device)}')

In [ ]:
# 读取测试图像并进行预测
# 读取图像并添加批量维度，转换为浮点数
X = torchvision.io.read_image('./banana.jpg').unsqueeze(0).float()
# 移除批量维度，调整维度顺序为 (高, 宽, 通道) 以便显示
img = X.squeeze(0).permute(1, 2, 0).long()

# 定义预测函数
def predict(X):
    """
    对输入图像进行目标检测预测
    参数:
        X: 输入图像张量
    返回:
        检测到的目标框及其置信度
    """
    net.eval()  # 设置为评估模式（关闭dropout等）
    # 前向传播得到锚框、类别预测和边界框预测
    anchors, cls_preds, bbox_preds = net(X.to(device))
    # 对类别预测应用softmax得到概率分布
    # permute(0, 2, 1): 调整维度顺序为 (batch, num_classes+1, num_anchors)
    cls_probs = F.softmax(cls_preds, dim=2).permute(0, 2, 1)
    # 非极大值抑制：去除重叠的检测框，只保留最佳的
    output = d2l.multibox_detection(cls_probs, bbox_preds, anchors)
    # 过滤掉背景（类别为-1的框）
    idx = [i for i, row in enumerate(output[0]) if row[0] != -1]
    return output[0, idx]  # 返回检测到的目标

output = predict(X)  # 执行预测

In [ ]:
# 显示检测结果
def display(img, output, threshold):
    """
    在图像上绘制检测框
    参数:
        img: 原始图像
        output: 检测结果，每行包含 [类别, 置信度, x_min, y_min, x_max, y_max]
        threshold: 置信度阈值，只显示置信度高于此值的检测框
    """
    d2l.set_figsize((5, 5))  # 设置图像大小
    fig = d2l.plt.imshow(img)  # 显示图像
    
    for row in output:  # 遍历每个检测结果
        score = float(row[1])  # 获取置信度
        if score < threshold:  # 如果置信度低于阈值，跳过
            continue
        h, w = img.shape[0:2]  # 获取图像高度和宽度
        # 将归一化的边界框坐标转换为像素坐标
        # row[2:6]: [x_min, y_min, x_max, y_max] (归一化坐标，范围0-1)
        # 乘以 (w, h, w, h): 转换为像素坐标
        bbox = [row[2:6] * torch.tensor((w, h, w, h), device=row.device)]
        # 在图像上绘制边界框，显示置信度，颜色为白色('w')
        d2l.show_bboxes(fig.axes, bbox, '%.2f' % score, 'w')

# 显示检测结果，只显示置信度大于0.9的检测框
display(img, output.cpu(), threshold=0.9)